In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/deep_learning_school/part_2/homework_1/
!ls

Mounted at /content/drive
/content/drive/MyDrive/deep_learning_school/part_2/homework_1
 data				     'SO_vectors_200.bin?download=1'
'[homework]simple_embeddings.ipynb'   stackoverflow_similar_questions.zip
 __MACOSX			      word2vec.model


In [145]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.5MB/s 
     |████████████████████████████████| 890kB 35.0MB/s 
     |████████████████████████████████| 3.2MB 22.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7ab77c432836b7d3ef18e1519c433ff54f598af9edac2202ea6ce69b7c7c5d83
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2021-02-20 15:20:26--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  14.0MB/s    in 92s     

2021-02-20 15:22:00 (15.0 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

In [5]:
wv_embeddings['aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa']

array([-1.55645888e-03, -2.16608308e-03, -4.14667855e-04, -1.25397841e-04,
       -1.08947197e-03, -8.99872277e-04,  3.45279026e-04, -1.54502958e-03,
        1.97099708e-03, -2.02103192e-03,  1.75641058e-03, -1.45361875e-03,
       -1.63643947e-03, -2.26348196e-03,  1.75440696e-03, -1.67901558e-03,
        2.42695049e-03,  5.51860256e-04, -2.00596335e-03, -8.70801043e-04,
        1.33796828e-03,  2.30281265e-03, -2.14449386e-03, -3.44436703e-04,
        2.11918005e-03, -1.68367254e-03, -1.69502699e-03, -1.06914504e-03,
       -5.24467498e-04,  3.34613032e-05, -2.33333348e-03,  6.62219478e-04,
       -6.08382215e-05,  1.05069997e-03,  2.49950425e-03,  2.41000741e-03,
       -2.53798265e-04, -1.39497546e-03, -1.69535016e-03,  2.44742911e-03,
        1.59941579e-03, -2.10532290e-03,  1.29508961e-03, -2.49234959e-03,
        2.28417129e-03,  5.75002108e-04,  1.63092604e-03, -1.54208601e-03,
       -1.36250118e-03,  4.70931467e-04,  3.23439745e-04, -9.81749385e-04,
       -1.40053569e-03, -

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
# '''your code'''

In [5]:
def most_simmilar(base_word='cat', search_word='dog', top_n=5, emb_dict=wv_embeddings):
  if word in emb_dict:
    top_similar = emb_dict.most_similar([emb_dict[base_word]], topn=top_n)
    top_similar = [w[0] for w in top_similar]
    
    # answer on question above
    if search_word in top_similar:
      print(f"Top similar words on '{base_word}' include '{search_word}'")
    else:
      print(f"Top similar words on '{base_word}' don't include '{search_word}'")
  else:
    print(f"Word: {word} is out of vocabulary! Please try another word")

In [6]:
most_simmilar('cat')

Top similar words on 'cat' include 'dog'


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [7]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [204]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
import spacy 
from transformers import BertTokenizer
from nltk import word_tokenize, ngrams, everygrams
# nlp = spacy.load("en_core_web_sm")
from spacy.lang.en import English
from nltk.tokenize import WordPunctTokenizer
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
spacy_tok = nlp.tokenizer
nltk_tokenizer = WordPunctTokenizer()
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class MyTokenizer:
    def __init__(self):
      pass

    def tokenize(self, text):
        # return re.findall('\w+', text)
        # return [x for x in re.split(r"([.,!?]+)?\s+", text) if x]
        return [str(tok.lemma_).lower() for tok in spacy_tok(text) if not tok.is_stop]
        # return [str(tok) for tok in spacy_tok(text) if not tok.is_stop]
        # return nltk_tokenizer.tokenize(text)
    def __call__(self, text):
      return self.tokenize(str(text))
    def word_piece(self, text):
      return [tok.replace("##", "") for tok in bert_tokenizer.tokenize(text)]


tokenizer = MyTokenizer()

In [262]:
from numpy.linalg import norm
def question_to_vec(question, embeddings, tokenizer=tokenizer, dim=200):
    """
        question: строка 
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    question = str(question)
    tokenized_question = tokenizer(question)
    # tokenized_question_vectors = np.array([embeddings[tok] for tok in tokenized_question if tok in embeddings])
    tokenized_question_vectors = []
    for tok in tokenized_question:
      if tok in embeddings:
        tokenized_question_vectors.append(embeddings[tok])
      else:
        unknown_words = [''.join(sub_tok) for sub_tok in list(everygrams(tok, 3))]
        # print(unknown_words)
        temp = []
        for unknown_word in unknown_words:
          if unknown_word in embeddings:
            temp.append(embeddings[unknown_word])
        if len(temp) > 0: 
          temp = np.mean(np.array(temp), axis=0)
          tokenized_question_vectors.append(temp)
    
    tokenized_question_vectors = np.array(tokenized_question_vectors)
    # tokenized_question_vectors = tokenized_question_vectors.astype(float)

    if len(tokenized_question_vectors) > 0:
      std = tokenized_question_vectors.std(-1, keepdims=True)
      eps = 1e-6
      tokenized_question_vectors = tokenized_question_vectors / (std + eps)
      # print(tokenized_question_vectors)
      question_vec = np.mean(tokenized_question_vectors, axis=0) 
      return question_vec
    return np.zeros((dim,)) 

In [205]:
list(everygrams("C++", 2, 3))

[('C', '+'), ('+', '+'), ('C', '+', '+')]

In [148]:
# What're hell??!
# wv_embeddings['ug']

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [258]:
'''your code'''
round(question_to_vec("I love neural networks", wv_embeddings)[2], 2)

-0.73

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

In [ ]:
"""
Пусть N = 1, rank_q′ = 2
Hits47 достигает максимума 1  
DCG1 минимум равен 0 при rank_q′ = 2
ОТВЕТ Максимум = 1 
"""

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [140]:
f"DCG@10 = {round(dcg_score([9], 10), 2)}"

'DCG@10 = 0.3'

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [101]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    dup_ranks = np.array(dup_ranks)
    hits_value = np.mean((dup_ranks <= k).astype(np.int))
    return hits_value    

In [102]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    eval_func = lambda rank_q: 1/np.log2(1 + rank_q) * int(rank_q <= k)

    dup_ranks = np.array([eval_func(rank_qi) for rank_qi in dup_ranks])
    dcg_value = np.mean(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
import pandas as pd

In [150]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],
                      ]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[i].index(q) + 1 for i, q in enumerate(copy_answers)]

# вычисляем метрику для разных k
# print(dup_ranks)
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [151]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!ls

 data				     'SO_vectors_200.bin?download=1'
'[homework]simple_embeddings.ipynb'   stackoverflow_similar_questions.zip
 __MACOSX			      word2vec.model


In [ ]:
!unzip stackoverflow_similar_questions.zip

Считайте данные.

In [152]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        line = str(line).split("\t")
        # line = line[:50]
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [153]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [61]:
len(validation_data)

3760

In [ ]:
# validation_data[0]

Размер нескольких первых строк

In [107]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [154]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [255]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    result = []
    question = question_to_vec(question, embeddings)
    cands = [question_to_vec(vec, embeddings) for vec in candidates]
    cosine_sim = 1 - cosine_similarity([question], cands)
    sim_index = np.argsort(cosine_sim)
    result = [(i, candidates[i]) for i in sim_index[0]]

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [175]:
# question_to_vec("converting string to list", wv_embeddings)

In [210]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [263]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [67]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')] #скрыт
            ]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

In [26]:
"""
Ответ 021
"""

'\nОтвет 021\n'

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [27]:
from tqdm.notebook import tqdm

In [264]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [265]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.450 | Hits@   1: 0.450
DCG@   5: 0.544 | Hits@   5: 0.627
DCG@  10: 0.565 | Hits@  10: 0.692
DCG@ 100: 0.610 | Hits@ 100: 0.909
DCG@ 500: 0.620 | Hits@ 500: 0.984
DCG@1000: 0.621 | Hits@1000: 1.000



### Эмбеддинги, обученные на корпусе похожих вопросов

In [266]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [267]:
words = [tokenizer(" ".join(line)) for i, line in enumerate(tqdm(train_data))]

In [268]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5,
                 workers=20).wv

In [269]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [270]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.436 | Hits@   1: 0.436
DCG@   5: 0.523 | Hits@   5: 0.601
DCG@  10: 0.551 | Hits@  10: 0.690
DCG@ 100: 0.596 | Hits@ 100: 0.908
DCG@ 500: 0.606 | Hits@ 500: 0.984
DCG@1000: 0.608 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Вывод\
1) Больше всего помогает принцип токенизации из spacy который основан на правилах и статистике, просто потому что он знает о языке больше чем старндартное разделение по пробелу.
Также несколько улучшила резутаты trigram'ы слов, потому как мы смогли вытащить больше информации из незнакомых слов или же просто обработали очепятки

2) Нормализация не дает существенных результатов, может быть только сотые доли, в теории это должно помочь когда словарь маленький, но на деле это помогает не сильно (лично мне)

3) Предобученные эмбединги справляются с задачей лучше, потому что видимо при их обучении использовалось больше данных, либо применялись другие методы токенизации

4) Потому что мы никак не учитываем контекст слов, их взяимосвязь.

5) В этом нам должны помочь контекстные эмбединги от BERT и siamese архитектура, по типу как сделано для определения похожих людей в CV

